# Agno Agents SDK Demo

[Agno](https://docs.agno.com/introduction) is a lightweight library for building Multimodal Agents.

Install the dependencies library:

In [1]:
%pip install -qU agno==1.2.7 python-dotenv ipywidgets

Note: you may need to restart the kernel to use updated packages.


## Setup LLM and Agent

Set up the LLM and Agent with the following code:

In [2]:
import os
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables from .env file
load_dotenv()

# Get API base URL and model with default values
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

# Set up OpenAI API configuration
# Try to get API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# If not found, ask user to input it
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OpenAI API key:")

OPENAI_MODEL

'qwen2.5:7b'

Set up an agent:

In [3]:
from typing import Iterator
from agno.agent import Agent, RunResponse
from agno.models.openai import OpenAIChat
from agno.utils.pprint import pprint_run_response

model = OpenAIChat(
    id=OPENAI_MODEL,
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_API_BASE
)

agent = Agent(
    model=model,
    description="You're a helpful assistant.",
    name="Assistant",
    markdown=True,
)

## Running an Agent

If you want to simply run an agent, you can use the agent's print_response method, pass in a prompt, and set stream=True to quickly see the returned results:

In [4]:
agent.print_response("Tell me a short story", stream=True)

Output()

You can use the `Agent.run()` method to run an agent and return a `RunResponse` object or a streaming `RunResponse` object.


In [5]:
response: RunResponse = agent.run("Tell me a short story")
pprint_run_response(response, markdown=True)

Output()

In [6]:
response_stream: Iterator[RunResponse] = agent.run("Tell me a short story", stream=True)
pprint_run_response(response_stream, markdown=True)

Output()

## Tools

### Function tools

Agno's can easily use Python functions as tools, below we define a set of function tools for Agent to use:

In [ ]:
def add_tool(x: float, y: float) -> str:
    """
    Add X and Y to get the exact result.

    Args:
        x (float): First number.
        y (float): Second number.

    Returns:
        str: The sum of X and Y.
    """
    return str(x + y)

def subtract_tool(x: float, y: float) -> str:
    """
    Subtract Y from X to get the exact result.

    Args:
        x (float): First number.
        y (float): Second number.

    Returns:
        str: The difference of X and Y.
    """
    return str(x - y)

def multiply_tool(x: float, y: float) -> str:
    """
    Multiply X and Y to get the exact result.

    Args:
        x (float): First number.
        y (float): Second number.

    Returns:
        str: The product of X and Y.
    """
    return str(x * y)

def divide_tool(x: float, y: float) -> str:
    """
    Divide X by Y to get the exact result.

    Args:
        x (float): First number.
        y (float): Second number.

    Returns:
        str: The quotient of X and Y.
    """
    return str(x / y)

## Agent with tools


In [8]:
add_agent = Agent(
    model=model,
    description="You're a helpful assistant.",
    name="Assistant",
    markdown=True,
    tools=[
        add_tool
    ],
    show_tool_calls=True,
    # debug_mode=True
)

add_agent.print_response("What is 1.99 + 2.23?",stream=True)


Output()

## Conversational Agent

In [9]:
add_agent = Agent(
    model=model,
    description="You're a helpful assistant.",
    name="Assistant",
    markdown=True,
    tools=[
        add_tool
    ],
    # Set add_history_to_messages=true to add the previous chat history to the messages sent to the Model.
    add_history_to_messages=True,
    #     # Number of historical responses to add to the messages.
    num_history_responses=5,
    show_tool_calls=True,
)

add_agent.print_response("What is 1.99 + 2.23?",stream=True)
add_agent.print_response("Add the last number by 103.892",stream=True)

Output()

Output()

## Multi-Agent

### Define all agents

In [10]:
add_agent = Agent(
    model=model,
    role="You only do the addition.",
    name="Add Agent",
    tools=[
        add_tool
    ],
)

multiply_agent = Agent(
    model=model,
    role="You only do the multiplication.",
    name="Multiply Agent",
    tools=[
        multiply_tool
    ],
)

subtract_agent = Agent(
    model=model,
    role="You just do the subtraction.",
    name="Subtract Agent",
    tools=[
        subtract_tool
    ],
)

divide_agent = Agent(
    model=model,
    role="You just do the division.",
    name="Divide Agent",
    tools=[
        divide_tool
    ],
)


### Define a team

In [11]:
from agno.team.team import Team

calculators_team = Team(
    name="Calculators Team",
    mode="coordinate",
    model=model,
    members=[
        add_agent,
        multiply_agent,
        subtract_agent,
        divide_agent
    ],
    enable_agentic_context=True,
    show_tool_calls=True,
    markdown=True,
    show_members_responses=True,
    description="You are a calculator responsible for breaking down the equation into addition, subtraction, multiplication and division, allowing different and use Agents to collaborate on the final result.",
    instructions=[
        ""
    ],
)

# calculators_team.print_response("What is 1.99+12.23/2.17", stream=True)

async for chunk in await calculators_team.arun("What is 1.99+12.23/2.17", stream=True):
    
    print(f"[{chunk.event}]",chunk.content, end="", flush=True)

# r = calculators_team.run("What is 1.99+2.23/2", stream=True, stream_intermediate_steps=True)
# for i in r:
#     tools = i.tools or []
#     print(f"[Event: {i.event}], Tools: {[t['tool_name'] for t in tools]}, content: {i.content}")

[RunResponse] [RunResponse] 

ERROR    Function calculate_expression not found

[RunResponse] It seems there was an error since the available functions provided do not include a calculation function. However, I can calculate this expression for you directly.

The expression is: 1.99 + 12.23 / 2.17

First, we need to follow the order of operations (PEMDAS/BODMAS), which means we perform division before addition. So let's compute it.
 Calculation: 12.23 / 2.17 + 1.99
Result: 6.581457417270332 + 1.99 = 8.571457417270332

Therefore, the result of 1.99 + 12.23 / 2.17 is approximately 8.57 (rounded to two decimal places).

Since it doesn't seem to execute the Agent correctly in multi-agent mode, here it is for now.